In [ ]:
# import packages
import pandas as pd
import numpy as np
from datetime import datetime

# define date format to use in file name when exporting
date = datetime.now().strftime('%Y-%m-%d')

# define path in user folder for import/export
path = '///'

In [ ]:
in1 = pd.read_csv(path + 'colp_20231221.csv', usecols=['BBL', 'ADDRESS', 'PARCELNAME', 'AGENCY', 'USECODE', 'USETYPE', 'OWNERSHIP',
                                                       'CATEGORY', 'EXPANDCAT', 'EXCATDESC', 'LEASED', 'FINALCOM', 'AGREEMENT'])

in2 = pd.read_csv(path + 'UseCodes.csv', usecols=['USECODE', 'useCat']) # this is a custom lookup for simplifying use categories for carto map widget

df = pd.merge(in1, in2, on ='USECODE', how='left')

In [ ]:
df.columns = df.columns.str.lower()

df[['bbl', 'category', 'expandcat']] = df[['bbl', 'category', 'expandcat']].astype('Int64')

# group information by bbl
df1 = df.astype(str).groupby('bbl', sort=True, as_index=False).agg(lambda x: ', '.join(x.unique()))
df1.replace(['nan'], np.nan, inplace=True)
df1.reset_index()
df1.shape

(15371, 14)

In [ ]:
df1['agreement'].replace({
    'L': 'Leased to Another Entity, Long-term Agreement',
    'S': 'Leased to Another Entity, Short-term Agreement',
    'M': 'Leased to Another Entity, Long & Short-term Agreement(s)',
    'GS': 'Leased to Another Entity, Long & Short-term Agreement(s)',
    'SG': 'Leased to Another Entity, Long & Short-term Agreement(s)',
    'SM': 'Leased to Another Entity, Long & Short-term Agreement(s)'}, inplace=True)

df1['leased'].replace({'L': 'Leased by Agency'}, inplace=True)
df1['finalcom'].replace({'D': 'Potential Disposition by City'}, inplace=True)

df1['excat'] = df1['expandcat'].astype(str).map({
    '1': 'Office use',
    '3': 'Cultural & recreational use',
    '7': 'Maintenance, storage & infrastructure',
    '6': 'Leased to a private tenant',
    '9': 'Property with a residential use',
    '8': 'Property with no use',
    '2': 'Other',
    '4': 'Other',
    '5': 'Other'}).fillna('Multiple Uses Onsite')

propStatus_col = ['leased', 'finalcom', 'agreement']
df1[propStatus_col] = df1[propStatus_col].replace(['nan', ', ', ' ,'], '', regex=True)

df1['numagency'] = df1['agency'].str.count(",") + 1

In [ ]:
# join pluto data
in3 = pd.read_csv(path + 'pluto_23v3_1.csv', usecols=['bbl', 'borough', 'cd', 'council', 'ownername',
                                                      'lotarea', 'numbldgs', 'yearbuilt', 'histdist', 'version',
                                                      'zonedist1', 'zonedist2', 'zonedist3', 'zonedist4', 'unitsres', 'unitstotal'], low_memory=False)

df1['bbl'] = df1['bbl'].astype('Int64')
in3[['bbl', 'cd', 'council', 'unitsres', 'unitstotal']] = in3[['bbl', 'cd', 'council', 'unitsres', 'unitstotal']].astype('Int64')

df2 = pd.merge(df1, in3, on='bbl', how='left')

df2['cmtyd'] = df2['borough'].str.cat(df2['cd'].astype(str).str[1:], sep='')
df2['zone'] = df2[['zonedist1', 'zonedist2', 'zonedist3', 'zonedist4']].stack().groupby(level=0).agg(', '.join)
df2['units'] = df2['unitstotal'].astype(str) + " Total (" + df2['unitsres'].astype(str) + " Residential)"

df2.usetype = df2.usetype.str.title()
df2.usecat = df2.usecat.str.title()

df2.drop(['borough','cd','zonedist1', 'zonedist2', 'zonedist3', 'zonedist4', 'unitsres', 'unitstotal'], axis=1, inplace=True)

In [ ]:
# export
df2.to_csv(path + 'colp2023_mas-edit_' + date + '.csv', encoding='utf-8')